##  2.9.2 Construction of machine learning models

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import shap
from sklearn.model_selection import cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, roc_auc_score, confusion_matrix, roc_curve, auc
from sklearn.preprocessing import label_binarize, StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from imblearn.over_sampling import SMOTE
from xgboost import XGBClassifier
import os
import statsmodels.api as sm  # For LOWESS fitting

# Step 1: Read the data
file_path = 'C:\\Users\\Lee66\\Desktop\\data.csv'
try:
    data = pd.read_csv(file_path, encoding='ISO-8859-1')
except UnicodeDecodeError:
    data = pd.read_csv(file_path, encoding='GBK')

# Step 2: Split the data into features and labels
X = data.drop(columns=['sample', 'Lable'])  # 'sample' and 'Lable' are dropped
y = data['Lable']

# Label encoding of target variable
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Step 3: Apply SMOTE for balancing the dataset
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y_encoded)

# Step 4: Standardize features
scaler = StandardScaler()
X_resampled_scaled = scaler.fit_transform(X_resampled)

# Step 5: Define machine learning models
models = {
    'Logistic Regression': LogisticRegression(max_iter=200),
    'SVM': SVC(probability=True),
    'KNN': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(max_depth=10),
    'Random Forest': RandomForestClassifier(n_estimators=50, max_depth=10, min_samples_leaf=4),
    'Naive Bayes': GaussianNB(),
    'MLP': MLPClassifier(max_iter=200, alpha=0.001),
    'Gradient Boosting': GradientBoostingClassifier(learning_rate=0.01, max_depth=5),
    'AdaBoost': AdaBoostClassifier(n_estimators=50, learning_rate=0.1),
    'XGBoost': XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', learning_rate=0.01, max_depth=5)
}

# Step 6: Initialize result storage
results = {}
roc_curves = {}

# Step 7: Perform cross-validation and store ROC curves
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

for name, model in models.items():
    print(f"Training {name} with cross-validation...")
    
    # Cross-validation prediction and probability
    y_pred = cross_val_predict(model, X_resampled_scaled, y_resampled, cv=cv, method='predict')
    y_prob = cross_val_predict(model, X_resampled_scaled, y_resampled, cv=cv, method='predict_proba')
    
    # Calculate evaluation metrics
    accuracy = accuracy_score(y_resampled, y_pred)
    f1 = f1_score(y_resampled, y_pred, average='weighted')
    recall = recall_score(y_resampled, y_pred, average='weighted')
    precision = precision_score(y_resampled, y_pred, average='weighted')
    auc_score = roc_auc_score(label_binarize(y_resampled, classes=np.arange(len(le.classes_))), y_prob, multi_class='ovr', average='weighted')
    
    # Store results
    results[name] = {
        'accuracy': accuracy,
        'f1_score': f1,
        'recall': recall,
        'precision': precision,
        'auc': auc_score,
        'confusion_matrix': confusion_matrix(y_resampled, y_pred)
    }
    
    # Compute ROC curves
    fpr, tpr, roc_auc = {}, {}, {}
    
    for i in range(len(le.classes_)):
        # ROC curve
        fpr[i], tpr[i], _ = roc_curve(label_binarize(y_resampled, classes=np.arange(len(le.classes_)))[:, i], y_prob[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])
    
    roc_curves[name] = (fpr, tpr, roc_auc)

# Step 8: Output the performance metrics
metrics_df = pd.DataFrame(results).T[['accuracy', 'f1_score', 'recall', 'precision', 'auc']]
print(metrics_df)

# Step 9: Visualize ROC curves
for name, (fpr, tpr, roc_auc) in roc_curves.items():
    plt.figure(figsize=(8, 6))
    for i in range(len(le.classes_)):
        plt.plot(fpr[i], tpr[i], label=f'Class {le.classes_[i]} (AUC = {roc_auc[i]:.2f})')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'ROC Curves for {name}')
    plt.legend(loc="lower right")
    plt.show()

# Step 10: Visualize confusion matrices
for name, result in results.items():
    cm = result['confusion_matrix']
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=le.classes_, yticklabels=le.classes_)
    plt.title(f'Confusion Matrix for {name}')
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.show()

# Step 11: Visualize AUC curves for all models in a single plot
plt.figure(figsize=(10, 8))
colors = ['b', 'g', 'r', 'c', 'm', 'y', 'k', 'orange', 'purple', 'pink']

for idx, (name, (fpr, tpr, roc_auc)) in enumerate(roc_curves.items()):
    plt.plot(fpr[0], tpr[0], color=colors[idx], lw=2, label=f'{name} (AUC = {roc_auc[1]:.2f})')

plt.plot([0, 1], [0, 1], 'k--', lw=2)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for All Models')
plt.legend(loc="lower right")
plt.show()

# Step 12: SHAP analysis for the best model 
best_model = models['XGBoost']  # Use XGBoost for SHAP analysis
best_model.fit(X_resampled, y_resampled)  # Train the model

# Calculate SHAP values
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_resampled)

# Step 13: Visualize SHAP summary plot
shap.summary_plot(shap_values, X_resampled, feature_names=X.columns)

# Step 14: SHAP Beeswarm plot
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_resampled, plot_type="bee swarm")

# Step 15: SHAP dependence plot with LOWESS
feature = 'Compound name'

# If shap_values is a list, process it as a numpy array
if isinstance(shap_values, list):
    shap_values = np.array(shap_values)

# Ensure shap_values is 2D for multi-class case
if shap_values.ndim > 2:
    shap_values_mean = shap_values[0][:, X_resampled.columns.get_loc(feature)]
else:
    shap_values_mean = shap_values[:, X_resampled.columns.get_loc(feature)]

# LOWESS fitting
plt.figure(figsize=(10, 6))
plt.scatter(X_resampled[feature], shap_values_mean, color='steelblue', alpha=0.6, label='SHAP values')

# Apply LOWESS for smoothing
lowess = sm.nonparametric.lowess(shap_values_mean, X_resampled[feature], frac=0.3)
x_lowess, y_lowess = zip(*lowess)
plt.plot(x_lowess, y_lowess, color='red', linewidth=2, label='LOWESS Curve')

# Find intersection points (where SHAP value crosses zero)
intersection_indices = np.where(np.diff(np.sign(y_lowess)))[0]
for idx in intersection_indices:
    plt.axvline(x=x_lowess[idx], color='blue', linestyle='--')

plt.xlabel(feature)
plt.ylabel('SHAP Value')
plt.title(f'SHAP Dependence Plot for {feature}')
plt.legend(loc="best")
plt.show()

# Save SHAP summary plot and Beeswarm plot as PDFs
plt.savefig(os.path.join(os.path.expanduser('~'), 'Desktop', 'SHAP_Summary_Plot)
